In [1]:
import cv2


In [2]:
import mediapipe as mp
import numpy as np
from scipy.signal import lfilter
import matplotlib.pyplot as plt

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
hands = mp_hands.Hands(max_num_hands = 2, static_image_mode = False,min_detection_confidence = 0.5)

In [5]:
# storing all the coordinates at different timesteps in a list
list_x = []
list_y = []
list_z = []
list_avg_x = []
list_avg_y = []
list_avg_z = []


In [6]:
# color and thickness of the lines
color = (255,0,0)
thickness = 2
radius = 2
# threshold after which we should start drawing
threshold = 4

# moving average buffer
k = 5


In [7]:
# canvas on which we are drawing
canvas = np.zeros([512,512,1],dtype=np.uint8)
display = canvas.copy()
cursor = canvas.copy()

In [8]:
toggle = 1

     
    

In [9]:
cap = cv2.VideoCapture(0)
count = 0
while cap.isOpened():
    success, image = cap.read()
    
        
    if not success:
        print("video capture unsuccessful")
        break
    
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    
    
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    key = cv2.waitKey(5)
    
    if key == 225:
        toggle = toggle^1
    elif key & 0xFF == ord('s'):
        break
    
    
        
    
    if results.multi_hand_landmarks:
        
        
        
            count = count+1
            hand_landmarks = results.multi_hand_landmarks[0]
        
            # add the coordinates to the list
            list_x.append(int(hand_landmarks.landmark[8].x*512))
            list_y.append(int(hand_landmarks.landmark[8].y*512))
            list_z.append(int(hand_landmarks.landmark[8].z*512))
            
            if(count == k):
                avg_x = sum(list_x[-k:])/k
                avg_y = sum(list_y[-k:])/k
                avg_z = sum(list_z[-k:])/k
            
                list_avg_x.append(int(avg_x))
                list_avg_y.append(int(avg_y))
                list_avg_z.append(int(avg_z))
                count = 0
            
        
            if(len(list_avg_x) > threshold and toggle == 1):
                # get the current coordinates as endpoint
                end_point = (list_avg_x[-1], list_avg_y[-1])
                # get the previous coordinates as the start point
                start_point = (list_avg_x[-2], list_avg_y[-2])
                # draw a line joining these points
                cv2.line(canvas,start_point, end_point,color, thickness)
                display = canvas.copy()
            
            elif(len(list_avg_x) > threshold and toggle == 0):
                
                # get the current coordinates as endpoint
                end_point = (list_avg_x[-1], list_avg_y[-1])
                cursor = canvas.copy()
                cv2.circle(cursor,end_point,radius,color,thickness)
                display = cursor.copy()
                
                
                
                
                
       
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
            
    # show the canvas and the webcam image   
    cv2.imshow("Detected Hands", image)
    cv2.imshow("Result", display)
    
    
    
  
hands.close()
cv2.destroyAllWindows()
cap.release()






In [10]:
print(list_avg_x)

[322, 319, 304, 303, 305, 310, 309, 306, 308, 312, 312, 308, 303, 301, 303, 296, 281, 259, 242, 239, 257, 281, 303, 320, 329, 330, 302, 275, 257, 246, 242, 242, 237, 231, 219, 194, 166, 142, 149, 157, 161, 159, 156, 149, 145, 135, 123, 115, 113, 114, 119, 127, 127, 127, 132, 139, 149, 163, 183, 197, 205, 206, 207, 194, 182, 169, 145, 130, 126, 128, 129, 129, 124, 119, 131, 157, 180, 194, 185, 190, 202, 192, 186, 216, 222, 212, 230, 267, 296, 312, 328, 344, 352, 350, 347, 336, 313, 294, 288, 294, 314, 338, 358, 372, 376, 366, 353, 371, 369, 360, 353, 343, 331, 320, 308, 302, 302]


In [11]:
print(list_avg_y)

[364, 274, 176, 101, 75, 59, 52, 50, 53, 55, 57, 55, 55, 54, 54, 54, 53, 56, 70, 90, 104, 105, 106, 119, 146, 185, 195, 183, 177, 174, 175, 172, 163, 157, 140, 117, 105, 85, 80, 81, 84, 90, 98, 89, 72, 84, 129, 172, 214, 228, 192, 148, 119, 102, 82, 69, 71, 93, 128, 172, 206, 221, 206, 157, 131, 137, 149, 158, 161, 167, 168, 148, 132, 134, 140, 145, 135, 131, 128, 132, 162, 212, 248, 248, 244, 217, 206, 206, 222, 258, 261, 233, 228, 226, 227, 228, 227, 233, 252, 267, 258, 247, 251, 268, 293, 287, 269, 275, 293, 307, 318, 320, 321, 318, 323, 324, 327]


In [12]:
print(count)

0


In [13]:
print(len(list_avg_x))

117
